In [1]:
# API KEY를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv
# API KEY 정보로드
load_dotenv()

True

In [2]:
import os
print(os.environ['GOOGLE_API_KEY'][:10]+'*'*10)

AIzaSyC06D**********


In [3]:
from langchain_teddynote import logging

logging.langsmith("Synthetic Data Generation")

LangSmith 추적을 시작합니다.
[프로젝트명]
Synthetic Data Generation


In [4]:
from langchain_community.document_loaders import PDFPlumberLoader

# 문서 로더 생성
# loader = PDFPlumberLoader("data/Prompt_Tuning.pdf")
loader = PDFPlumberLoader("data/Domain-specialized-LLM-Financial-fine-tuning-and-utilization-method-using-Mistral-7B.pdf")

# 문서 로딩
docs = loader.load()

# 목차, 끝 페이지 제외
docs = docs[3:-1]

# 문서의 페이지수
len(docs)

25

In [5]:
docs[0].metadata

{'source': 'data/Domain-specialized-LLM-Financial-fine-tuning-and-utilization-method-using-Mistral-7B.pdf',
 'file_path': 'data/Domain-specialized-LLM-Financial-fine-tuning-and-utilization-method-using-Mistral-7B.pdf',
 'page': 3,
 'total_pages': 29,
 'Author': 'MS',
 'CreationDate': "D:20240328014510+09'00'",
 'Creator': 'PScript5.dll Version 5.2.2',
 'ModDate': "D:20240328014510+09'00'",
 'Producer': 'Acrobat Distiller 9.3.2 (Windows)',
 'Title': '<C1F6B4C9C1A4BAB8203330B1C731C8A35FC5EBBABB2E687770>',
 'rgid': 'PB:379341632_AS:11431281232126100@1711621456089'}

In [6]:
import re
for doc in docs:
	text_org = doc.page_content
	clean_text = re.split(r'\U000f080f', text_org)[-1]
	doc.page_content = clean_text


In [7]:
# metadata 설정(filename 이 존재해야 함)
for doc in docs:
    doc.metadata["filename"] = doc.metadata["source"]

In [8]:
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_google_genai import ChatGoogleGenerativeAI, GoogleGenerativeAIEmbeddings
from langchain_ollama import OllamaEmbeddings, ChatOllama

# 데이터셋 생성기
# llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash")
llm = ChatOpenAI(model="gpt-4o-mini")
# llm = ChatOllama(model="dnotitia/dna")
# llm = ChatOllama(model="llama3.2")


# 문서 임베딩
# embeddings = GoogleGenerativeAIEmbeddings(model="models/text-embedding-004")
# embeddings = OllamaEmbeddings(model="bge-m3")
embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")


In [9]:
from ragas.testset import TestsetGenerator
from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper
# LangChain의 gemini 모델을 LangchainLLMWrapper로 감싸 Ragas와 호환되게 만듭니다.
generator_llm = LangchainLLMWrapper(llm)

# ragas_embeddings 생성
ragas_embeddings = LangchainEmbeddingsWrapper(embeddings)


c:\Users\최인규\Documents\RAG_practice\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [10]:
from ragas.testset.graph import KnowledgeGraph
from ragas.testset.graph import Node, NodeType

kg = KnowledgeGraph()
for doc in docs:
    kg.nodes.append(
        Node(
            type=NodeType.DOCUMENT,
            properties={"page_content": doc.page_content, "document_metadata": doc.metadata}
        )
    )

In [11]:
kg

KnowledgeGraph(nodes: 25, relationships: 0)

### Transform 적용으로 노드 재생성 및 엣지(relationships) 생성

In [12]:
from ragas.testset.transforms import default_transforms, apply_transforms


# define your LLM and Embedding Model
# here we are using the same LLM and Embedding Model that we used to generate the testset
transformer_llm = generator_llm
embedding_model = ragas_embeddings

trans = default_transforms(documents=docs, llm=transformer_llm, embedding_model=embedding_model)


In [13]:
trans

[HeadlinesExtractor(name='HeadlinesExtractor', filter_nodes=<function default_transforms.<locals>.<lambda> at 0x0000026DF21E16C0>, llm=LangchainLLMWrapper(langchain_llm=ChatOpenAI(...)), merge_if_possible=True, max_token_limit=32000, tokenizer=<Encoding 'o200k_base'>, property_name='headlines', prompt=HeadlinesExtractorPrompt(instruction=Extract the most important max_num headlines from the given text that can be used to split the text into independent sections.Focus on Level 2 and Level 3 headings., examples=[(TextWithExtractionLimit(text='                Introduction\n                Overview of the topic...\n\n                Main Concepts\n                Explanation of core ideas...\n\n                Detailed Analysis\n                Techniques and methods for analysis...\n\n                Subsection: Specialized Techniques\n                Further details on specialized techniques...\n\n                Future Directions\n                Insights into upcoming trends...\n\n    

In [50]:
apply_transforms(kg, trans)

Applying HeadlineSplitter:   0%|          | 0/25 [00:00<?, ?it/s]           unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
Applying SummaryExtractor:  43%|████▎     | 20/46 [00:06<00:07,  3.43it/s]Property 'summary' already exists in node '1ccf5c'. Skipping!
Property 'summary' already exists in node '876026'. Skipping!
Applying SummaryExtractor:  57%|█████▋    | 26/46 [00:07<00:05,  3.76it/s]Property 'summary' already exists in node 'c5f3ca'. Skipping!
Property 'summary' already exists in node '70701d'. Skipping!
Applying SummaryExtractor:  74%|███████▍  | 34/46 [00:09<00:02,  4.95it/s]Property 'summary' already exists in node '5a06f5'. Skipping!
Property 'summary' already exists in node 'b549a7'. Skipping!
Applying SummaryExtractor:  78%|███████▊  | 36/46 [00:09<00:01,  6.73it/s]Property 'summary' already exists in node '43fb9d'. Skipping!
Property 'summary' already exists in node 

In [51]:
kg

KnowledgeGraph(nodes: 48, relationships: 1035)

In [52]:
kg.save("knowledge_graph.json")

In [14]:
loaded_kg = KnowledgeGraph.load("knowledge_graph.json")
loaded_kg

KnowledgeGraph(nodes: 48, relationships: 1035)

In [15]:
from ragas.testset.synthesizers import default_query_distribution
query_distribution = default_query_distribution(generator_llm)

In [16]:
for query, _ in query_distribution:
    prompts = await query.adapt_prompts("korean", llm=generator_llm)
    query.set_prompts(**prompts)

In [17]:
len(query_distribution)

3

In [18]:
from ragas.testset.synthesizers.single_hop.specific import (
    SingleHopSpecificQuerySynthesizer,
)

distribution = [
    (SingleHopSpecificQuerySynthesizer(llm=generator_llm), 1.0),
]

for query, _ in distribution:
    prompts = await query.adapt_prompts("korean", llm=generator_llm)
    query.set_prompts(**prompts)

In [19]:
query_distribution_change_ratio = [(dist[0], ratio)for dist, ratio in zip(query_distribution, [0.6,0.2,0.2])]
query_distribution_single = distribution

In [20]:
query_distribution_single

[(SingleHopSpecificQuerySynthesizer(name='single_hop_specifc_query_synthesizer', llm=LangchainLLMWrapper(langchain_llm=ChatOpenAI(...)), generate_query_reference_prompt=QueryAnswerGenerationPrompt(instruction=Generate a single-hop query and answer based on the specified conditions (persona, term, style, length) and the provided context. Ensure the answer is entirely faithful to the context, using only the information directly from the provided context.### Instructions:
  1. **Generate a Query**: Based on the context, persona, term, style, and length, create a question that aligns with the persona's perspective and incorporates the term.
  2. **Generate an Answer**: Using only the content from the provided context, construct a detailed answer to the query. Do not add any information not included in or inferable from the context.
  , examples=[(QueryCondition(persona=Persona(name='소프트웨어 엔지니어', role_description='코딩 모범 사례와 시스템 설계에 집중합니다.'), term='마이크로서비스', query_style='형식적', query_length='

In [21]:
from ragas.testset import TestsetGenerator

generator = TestsetGenerator(llm=generator_llm, embedding_model=embedding_model, knowledge_graph=loaded_kg)

In [22]:
from ragas.testset.persona import generate_personas_from_kg
from ragas.testset.graph import KnowledgeGraph

if generator.persona_list is None:
    personas = generate_personas_from_kg(
        llm=generator.llm,
        kg=generator.knowledge_graph,
        num_personas=3,  # 원하는 페르소나 개수
        callbacks=None   # 필요 시 콜백을 추가
    )
    generator.persona_list = personas
else:
    # 이미 페르소나가 있으면 섞어줍니다.
    import random
    random.shuffle(generator.persona_list)
    personas = generator.persona_list

print("생성된 페르소나:", personas)

Generating personas: 100%|██████████| 3/3 [00:01<00:00,  2.58it/s]

생성된 페르소나: [Persona(name='Financial Data Scientist', role_description='Specializes in analyzing financial data using advanced algorithms to ensure compliance, security, and accurate predictions.'), Persona(name='Financial Data Scientist', role_description='Specializes in developing algorithms and models for financial data analysis, ensuring security, compliance, and accuracy in predictions.'), Persona(name='Financial Data Scientist', role_description='Specializes in applying machine learning algorithms to analyze and predict financial trends, while ensuring data privacy and compliance.')]


In [29]:
# 필요한 모듈 및 함수 임포트
from ragas.testset.synthesizers.utils import calculate_split_values
from ragas.executor import Executor
from ragas.callbacks import new_group

# 테스트에 필요한 변수 설정
testset_size = 10   # 총 생성할 시나리오 샘플 수
num_personas = 3    # 사용할 페르소나 수
batch_size = None   # 배치 사이즈 (필요에 따라 설정)
run_config = None   # 실행 설정 (필요시 지정)
callbacks = []      # 빈 콜백 리스트 (원하는 콜백이 있으면 추가)

# 테스트셋 생성의 최상위 그룹 생성 (시나리오 생성 그룹의 상위 그룹)
RAGAS_TESTSET_GENERATION_GROUP_NAME = "ragas testset generation"
testset_generation_rm, testset_generation_grp = new_group(
    name=RAGAS_TESTSET_GENERATION_GROUP_NAME,
    inputs={"testset_size": testset_size},
    callbacks=callbacks,
)


# 전체 시나리오 샘플 수를 각 시나리오에 할당할 개수로 분할
splits, _ = calculate_split_values(
    [prob for _, prob in query_distribution_single],
    testset_size
)

# 시나리오 생성 그룹 생성 (콜백 체인 상위에 연결)
scenario_generation_rm, scenario_generation_grp = new_group(
    name="Scenario Generation",
    inputs={"splits": splits},
    callbacks=testset_generation_grp,
)

# Executor 객체를 생성하여 시나리오 생성 작업을 제출
exec = Executor(
    desc="Generating Scenarios",
    raise_exceptions=True,
    run_config=run_config,
    keep_progress_bar=False,
    batch_size=batch_size,
)

# 각 시나리오에 대해 generate_scenarios 함수를 호출하여 작업 제출
for i, (scenario, _) in enumerate(query_distribution_single):
    exec.submit(
        scenario.generate_scenarios,
        n=splits[i],
        knowledge_graph=generator.knowledge_graph,
        persona_list=generator.persona_list[:num_personas],
        callbacks=scenario_generation_grp,
    )

# 작업 결과를 수집하고 에러가 있으면 처리
try:
    scenario_sample_list = exec.results()
except Exception as e:
    scenario_generation_rm.on_chain_error(e)
    raise e
else:
    scenario_generation_rm.on_chain_end(outputs={"scenario_sample_list": scenario_sample_list})

# 생성된 시나리오 샘플 리스트 출력
print("생성된 시나리오 샘플 리스트:", scenario_sample_list)


Generating Scenarios: 100%|██████████| 1/1 [01:05<00:00, 65.16s/it]

생성된 시나리오 샘플 리스트: [[SingleHopScenario(
nodes=1
term=
persona=name='Financial Data Scientist' role_description='Specializes in analyzing financial data using advanced algorithms to ensure compliance, security, and accurate predictions.'
style=QueryStyle.PERFECT_GRAMMAR
length=QueryLength.MEDIUM), SingleHopScenario(
nodes=1
term=
persona=name='Financial Data Scientist' role_description='Specializes in analyzing financial data using advanced algorithms to ensure compliance, security, and accurate predictions.'
style=QueryStyle.PERFECT_GRAMMAR
length=QueryLength.SHORT)]]


In [30]:
scenario_sample_list

[[SingleHopScenario(
  nodes=1
  term=
  persona=name='Financial Data Scientist' role_description='Specializes in analyzing financial data using advanced algorithms to ensure compliance, security, and accurate predictions.'
  style=QueryStyle.PERFECT_GRAMMAR
  length=QueryLength.MEDIUM),
  SingleHopScenario(
  nodes=1
  term=
  persona=name='Financial Data Scientist' role_description='Specializes in analyzing financial data using advanced algorithms to ensure compliance, security, and accurate predictions.'
  style=QueryStyle.PERFECT_GRAMMAR
  length=QueryLength.SHORT)]]

In [33]:
from ragas.executor import Executor
from ragas.callbacks import new_group

# sample_generation 콜백 그룹 생성
sample_generation_rm, sample_generation_grp = new_group(
    name="Sample Generation",
    inputs={"scenario_sample_list": scenario_sample_list},
    callbacks=[],
)

# Executor 객체 생성
exec = Executor(
    desc="Generating Samples",
    raise_exceptions=True,
    run_config=None,   # 필요에 따라 실행 설정 지정
    keep_progress_bar=True,
    batch_size=None,   # 배치 크기 설정 (없으면 None)
)

# 평가 샘플 생성 시 synthesizer 객체의 generate_sample 메서드 사용
additional_testset_info = []
for i, (synthesizer, _) in enumerate(query_distribution_single):
    for scenario in scenario_sample_list[i]:
        exec.submit(
            synthesizer.generate_sample,  # synthesizer 객체의 generate_sample 호출
            scenario=scenario,            # 해당 시나리오 객체를 인자로 전달
            callbacks=sample_generation_grp,
        )
        additional_testset_info.append({"synthesizer_name": synthesizer.name})

# 작업 결과 수집
try:
    eval_samples = exec.results()
except Exception as e:
    sample_generation_rm.on_chain_error(e)
    raise e
else:
    sample_generation_rm.on_chain_end(outputs={"eval_samples": eval_samples})

print("생성된 평가 샘플:", eval_samples)


Generating Samples: 100%|██████████| 2/2 [00:04<00:00,  2.01s/it]

생성된 평가 샘플: [SingleTurnSample(user_input='금융 데이터의 특성을 고려할 때, LLM 파인튜닝에서 어떤 요소를 중점적으로 다루어야 합니까?', retrieved_contexts=None, reference_contexts=['\n데이터의 다양성을 높이기 위해 데이터 증 요하다. 금융 데이터는 다음과 같은 특성을 가지\n강을 활용한다. 문장 내 단어 순서 변경, 동 고 있다.\n의어 삽입 등의 기법을 사용하여 모델이\n1) 금융 전문 용어의 사용: 금융 데이터에는\n다양한 문맥을 학습하도록 도움을 준다.\n다양한 전문 용어가 사용된다. LLM이 이\n6) 실험 로깅 및 모니터링: TensorBoard나 원 러한 전문 용어를 이해하고 인식할 수 있\n하는 로깅 도구를 사용하여 학습 중 성능 도록 해야 한다. 예를 들어, “주가 하락”,\n을 모니터링하는데 학습 중에 중요한 지표 “환율 상승”, “금리 인상” 등이 있다. 또한\n들을 로깅하여 모델의 성능을 실시간으로 금융 상품에는 “적립식 펀드”, “변액 보험”,\n확인하고, 학습 과정에서 발생한 문제를 신 “대출” 등 다양하게 있다. 그리고 금융 상\n속하게 파악할 수 있다. 품에는 다양한 특성이 있다. 예를 들어, “적\n립식 펀드는 장기 투자에 적합하다.”, “변\n7) 하이퍼파라미터 튜닝: 하이퍼파라미터 최\n액 보험은 저축과 보험의 기능을 모두 가\n적화 도구를 활용하여 자동으로 최적의 하이\n지고 있다.”, “대출은 부채를 의미한다.” 등\n퍼파라미터 조합 찾기를 하게 되는데 Grid\n이 있다. LLM이 이러한 전문 용어를 이해\nSearch 또는 Random Search와 같은 튜닝\n하고 인식할 수 있도록, 사전 또는 어휘 사\n기법을 사용하여 하이퍼파라미터를 조정하\n전을 통해 전문 용어 및 금융 상품에 대한\n면 모델의 성능을 극대화할 수 있다.\n지식을 학습해야 한다.\n이렇게 모델 파인튜닝 실행 환경 설정은 실험\n2) 숫자의 사용: 금융 데이터에는 다양한 숫자\

In [42]:
scenario_sample_list[0]

[SingleHopScenario(
 nodes=1
 term=
 persona=name='Financial Data Scientist' role_description='Specializes in analyzing financial data using advanced algorithms to ensure compliance, security, and accurate predictions.'
 style=QueryStyle.PERFECT_GRAMMAR
 length=QueryLength.MEDIUM),
 SingleHopScenario(
 nodes=1
 term=
 persona=name='Financial Data Scientist' role_description='Specializes in analyzing financial data using advanced algorithms to ensure compliance, security, and accurate predictions.'
 style=QueryStyle.PERFECT_GRAMMAR
 length=QueryLength.SHORT)]

In [34]:
eval_samples

[SingleTurnSample(user_input='금융 데이터의 특성을 고려할 때, LLM 파인튜닝에서 어떤 요소를 중점적으로 다루어야 합니까?', retrieved_contexts=None, reference_contexts=['\n데이터의 다양성을 높이기 위해 데이터 증 요하다. 금융 데이터는 다음과 같은 특성을 가지\n강을 활용한다. 문장 내 단어 순서 변경, 동 고 있다.\n의어 삽입 등의 기법을 사용하여 모델이\n1) 금융 전문 용어의 사용: 금융 데이터에는\n다양한 문맥을 학습하도록 도움을 준다.\n다양한 전문 용어가 사용된다. LLM이 이\n6) 실험 로깅 및 모니터링: TensorBoard나 원 러한 전문 용어를 이해하고 인식할 수 있\n하는 로깅 도구를 사용하여 학습 중 성능 도록 해야 한다. 예를 들어, “주가 하락”,\n을 모니터링하는데 학습 중에 중요한 지표 “환율 상승”, “금리 인상” 등이 있다. 또한\n들을 로깅하여 모델의 성능을 실시간으로 금융 상품에는 “적립식 펀드”, “변액 보험”,\n확인하고, 학습 과정에서 발생한 문제를 신 “대출” 등 다양하게 있다. 그리고 금융 상\n속하게 파악할 수 있다. 품에는 다양한 특성이 있다. 예를 들어, “적\n립식 펀드는 장기 투자에 적합하다.”, “변\n7) 하이퍼파라미터 튜닝: 하이퍼파라미터 최\n액 보험은 저축과 보험의 기능을 모두 가\n적화 도구를 활용하여 자동으로 최적의 하이\n지고 있다.”, “대출은 부채를 의미한다.” 등\n퍼파라미터 조합 찾기를 하게 되는데 Grid\n이 있다. LLM이 이러한 전문 용어를 이해\nSearch 또는 Random Search와 같은 튜닝\n하고 인식할 수 있도록, 사전 또는 어휘 사\n기법을 사용하여 하이퍼파라미터를 조정하\n전을 통해 전문 용어 및 금융 상품에 대한\n면 모델의 성능을 극대화할 수 있다.\n지식을 학습해야 한다.\n이렇게 모델 파인튜닝 실행 환경 설정은 실험\n2) 숫자의 사용: 금융 데이터에는 다양한 숫자\n의 효율성 및 모델

In [70]:
testset = generator.generate(testset_size=10, query_distribution=query_distribution_single, with_debugging_logs=True)
# testset = generator.generate(testset_size=10, query_distribution=query_distribution_change_ratio)



Generating Scenarios: 100%|██████████| 1/1 [00:42<00:00, 42.90s/it]
Generating Samples: 0it [00:00, ?it/s]


In [71]:
testset.to_pandas()

""


In [72]:
testset

Testset(samples=[])

In [65]:
# 지식 그래프의 노드 속성 확인
node_properties = set()
for node in loaded_kg.nodes:
    for prop in node.properties:
        node_properties.add(prop)
print(f"노드 속성: {node_properties}")

# 노드 수 확인
print(f"노드 수: {len(loaded_kg.nodes)}")

노드 속성: {'document_metadata', 'headlines', 'page_content', 'summary', 'summary_embedding'}
노드 수: 48


In [ ]:
 from ragas.testset import TestsetGenerator

generator = TestsetGenerator(
    llm=generator_llm, embedding_model=generator_embeddings, persona_list=personas
)

In [ ]:
dataset = generator.generate_with_langchain_docs(
    docs[:],
    testset_size=5,
    transforms=transforms,
    query_distribution=distribution,
)

## 지식그래프 생성

### 기본 Node 생성
- 각 문서당 1개의 노드 

In [ ]:
from ragas.testset.graph import KnowledgeGraph
from ragas.testset.graph import Node, NodeType

kg = KnowledgeGraph()
for doc in docs:
    kg.nodes.append(
        Node(
            type=NodeType.DOCUMENT,
            properties={"page_content": doc.page_content, "document_metadata": doc.metadata}
        )
    )

- default_transforms에 포함되어 있는 내용
- 

In [ ]:
default_transforms

In [14]:
loaded_kg.nodes[0].get_property("summary")

'이 논문은 Mistral 7B를 활용한 금융 분야의 LLM 파인튜닝 및 활용 방법을 다룬다. LLM은 자연어 처리(NLP)에서 중요한 발전을 이루었으며, ChatGPT와 같은 모델이 많은 관심을 받고 있다. LLM은 대량의 텍스트 데이터를 학습하여 다양한 자연어 태스크에 적용할 수 있는 강력한 표현을 학습한다. 금융 분야에서는 고객 응대 챗봇을 통해 최신 정보 제공의 중요성이 강조되고 있다. 그러나 LLM의 정보 제한성과 환각 문제는 여전히 도전 과제로 남아 있다. 이를 해결하기 위해 새로운 데이터로의 파인튜닝과 프롬프트 콘텍스트에 정보를 삽입하는 방법이 제안되었다. RAG 모델이 대안으로 제시되며, 정보 검색을 통해 LLM에 필요한 정보를 전달하는 방식도 구현되고 있다. 이러한 기술들은 AI의 딥러닝 기반으로 생성형 AI 서비스를 가능하게 한다.'

In [10]:
from ragas.testset.synthesizers import default_query_distribution

query_distribution = default_query_distribution(generator_llm)

In [15]:
query_distribution_onlysingle = [(query_distribution[0][0], 1.0)]

In [24]:
prompts = query_type.get_prompts()
query_generation = prompts['query_answer_generation_prompt']
print(type(query_generation))
print(query_generation)

# print(prompts['themes_personas_matching_prompt'])



<class 'ragas.testset.synthesizers.single_hop.prompts.QueryAnswerGenerationPrompt'>
QueryAnswerGenerationPrompt(
  "name": "query_answer_generation_prompt",
  "instruction": "Generate a single-hop query and answer based on the specified conditions (persona, term, style, length) and the provided context. Ensure the answer is entirely faithful to the context, using only the information directly from the provided context.### Instructions:\n1. **Generate a Query**: Based on the context, persona, term, style, and length, create a question that aligns with the persona's perspective and incorporates the term.\n2. **Generate an Answer**: Using only the content from the provided context, construct a detailed answer to the query. Do not add any information not included in or inferable from the context.\n",
  "examples": [
    [
      {
        "persona": {
          "name": "Software Engineer",
          "role_description": "Focuses on coding best practices and system design."
        },
       

In [17]:
from ragas.testset import TestsetGenerator

generator = TestsetGenerator(llm=generator_llm, embedding_model=ragas_embeddings, knowledge_graph=loaded_kg)

In [18]:

testset = generator.generate(testset_size=3, query_distribution=query_distribution_onlysingle, num_personas=1)

Generating Samples:  33%|███▎      | 1/3 [02:04<04:09, 124.75s/it]


OutputParserException: Invalid json output: I am trying to understand how Large Language Models (LLMs) and Chatbots work, especially in the context of financial services. Can you explain it in simple terms?
For troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/OUTPUT_PARSING_FAILURE 

Prompt fix_output_format failed to parse output: The output parser failed to parse the output including retries.
Prompt fix_output_format failed to parse output: The output parser failed to parse the output including retries.
Prompt fix_output_format failed to parse output: The output parser failed to parse the output including retries.
Prompt query_answer_generation_prompt failed to parse output: The output parser failed to parse the output including retries.


In [ ]:
test_df = testset.to_pandas()

In [12]:
generator = TestsetGenerator(llm = generator_llm, embedding_model=ragas_embeddings)
dataset = generator.generate_with_langchain_docs(docs, testset_size=10, with_debugging_logs=True, raise_exceptions=False)

Applying HeadlinesExtractor:   0%|          | 0/11 [00:00<?, ?it/s]

Applying SummaryExtractor:   0%|          | 0/22 [00:00<?, ?it/s]           Retrying langchain_google_genai.chat_models._achat_with_retry.<locals>._achat_with_retry in 2.0 seconds as it raised ResourceExhausted: 429 Resource has been exhausted (e.g. check quota)..
Retrying langchain_google_genai.chat_models._achat_with_retry.<locals>._achat_with_retry in 2.0 seconds as it raised ResourceExhausted: 429 Resource has been exhausted (e.g. check quota)..
Retrying langchain_google_genai.chat_models._achat_with_retry.<locals>._achat_with_retry in 2.0 seconds as it raised ResourceExhausted: 429 Resource has been exhausted (e.g. check quota)..
Retrying langchain_google_genai.chat_models._achat_with_retry.<locals>._achat_with_retry in 2.0 seconds as it raised ResourceExhausted: 429 Resource has been exhausted (e.g. check quota)..
Retrying langchain_google_genai.chat_models._achat_with_retry.<locals>._achat_with_retry in 2.0 seconds as it raised ResourceExhausted: 429 Resource has been exhausted 

In [13]:
# 생성된 테스트셋을 pandas DataFrame으로 변환
test_df = dataset.to_pandas()
test_df

,user_input,reference_contexts,reference,synthesizer_name
0,What learning rate was used when tuning the pu...,[modelthatwecanreuseforprompttuningacross ingr...,When tuning the public T5.1.1 checkpoints on S...,single_hop_specifc_query_synthesizer
1,"As a machine learning researcher, can you elab...",[100\n90\n80\n70\n60\n50\n108 109 1010\nModel ...,Increasing to 20+ tokens generally confers a l...,single_hop_specifc_query_synthesizer
2,Considering my expertise in machine lerning an...,[the class based initialization performs best....,The context indicates that pre-training object...,single_hop_specifc_query_synthesizer
3,Considering the differences between prompt tun...,[ateverytransformerlayer. Thisisakintolearning...,Adapters modify the actual function that acts ...,single_hop_specifc_query_synthesizer
4,Wat is teh accurecy of teh BoolQ model?,[Train Eval Tuning Accuracy F1 Dataset Metric ...,"The BoolQ model has an accuracy of 91.1, 91.3,...",single_hop_specifc_query_synthesizer


In [14]:
test_df.shape

(5, 4)

## deeper